<a href="https://colab.research.google.com/github/yuhao831068/One_piece_character_classifier/blob/main/transfer_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import zipfile
import torch
from torch.utils.data import Dataset
from pathlib import Path
from PIL import Image
import requests

In [2]:
url ="https://firebasestorage.googleapis.com/v0/b/grandmacan-2dae4.appspot.com/o/ML_data%2Fone_piece_full.zip?alt=media&token=937656fd-f5c1-44f5-b174-1e2d590b8ef3"

with open('one_piece_full.zip', 'wb') as f:
  req = requests.get(url)
  f.write(req.content)

import zipfile

with zipfile.ZipFile('one_piece_full.zip', 'r') as zip_file:
  zip_file.extractall('one_piece_full')

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [4]:
def train_step(dataloader, model, cost_fn, optimizer, device):

  train_cost = 0
  train_acc = 0

  for batch, (x, y) in enumerate(dataloader):
    x = x.to(device)
    y = y.to(device )

    model.train()

    y_pred = model(x)

    cost = cost_fn(y_pred, y)
    train_cost += cost
    train_acc += (y_pred.argmax(dim=1)==y).sum() / len(y) * 100

    optimizer.zero_grad()

    cost.backward()

    optimizer.step()


  train_cost /= len(dataloader)
  train_acc /= len(dataloader)
  print(f'Train Cost: {train_cost:.4f}, Train Acc:{train_acc:.2f}')

def test_step(dataloader, model, cost_fn, device):

  test_cost = 0
  test_acc = 0
  model.eval()
  with torch.inference_mode():
    for x, y in dataloader:
      x = x.to(device)
      y = y.to(device )

      test_pred = model(x)
      test_cost +=  cost_fn(test_pred, y)
      test_acc += (test_pred.argmax(dim=1)==y).sum() / len(y) * 100

    test_cost /= len(dataloader)
    test_acc /= len(dataloader)
    print(f'Test Cost:{test_cost:.4f}, Test Acc:{test_acc:.2f}')

In [5]:
from torchvision import transforms

class ImageDataset(Dataset):
  def __init__(self, root, train, transform=None):
    if train:
      image_root = Path(root) / 'train'
    else:
      image_root = Path(root) / 'test'


    self.paths = [i for i in image_root.rglob('*') if i.is_file()]
    self.transform = transform

    with open(Path(root) / 'classnames.txt','r') as f:
      lines = f.readlines()
      classes = []
      for line in lines:
        stripped_line = line.strip()
        classes.append(stripped_line)
        self.classes = classes

  def __getitem__(self, index):
    img = Image.open(self.paths[index]).convert('RGB')
    class_name = self.paths[index].parent.name
    class_idx = self.classes.index(class_name)

    if self.transform:
      return self.transform(img), class_idx
    else:
      return img, class_idx

  def __len__(self):
    return len(self.paths)

In [6]:
import torchvision

weights = torchvision.models.EfficientNet_B1_Weights.DEFAULT
model = torchvision.models.efficientnet_b1(weights=weights)

model.to(device)

Downloading: "https://download.pytorch.org/models/efficientnet_b1-c27df63c.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b1-c27df63c.pth
100%|██████████| 30.1M/30.1M [00:00<00:00, 84.4MB/s]


EfficientNet(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): SiLU(inplace=True)
    )
    (1): Sequential(
      (0): MBConv(
        (block): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
            (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): SiLU(inplace=True)
          )
          (1): SqueezeExcitation(
            (avgpool): AdaptiveAvgPool2d(output_size=1)
            (fc1): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
            (fc2): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
            (activation): SiLU(inplace=True)
            (scale_activation): Sigmoid()
          )
          (2): Conv2dNormActivat

In [7]:
#檢查模型如何將原本資料集做轉換
efficientnet_b1_transforms = weights.transforms()

In [8]:
train_dataset = ImageDataset(root='one_piece_full',
             train=True,
             transform=efficientnet_b1_transforms)

test_dataset = ImageDataset(root='one_piece_full',
             train=False,
             transform=efficientnet_b1_transforms)

In [9]:
from torch.utils.data import DataLoader

BATCH_SIZE = 16

train_dataloader = DataLoader(dataset = train_dataset,
                              batch_size = BATCH_SIZE,
                              shuffle = True)

test_dataloader = DataLoader(dataset = test_dataset,
                              batch_size = BATCH_SIZE,
                              shuffle = False)

In [10]:
len(train_dataloader), len(test_dataloader)

(189, 47)

In [11]:
!pip install torchinfo
from torchinfo import summary

In [ ]:
# 把架構列出來看
summary(model=model,
        input_size=(16,3,64,64),
        col_names=['input_size','output_size','num_params','trainable'],
        row_settings=['var_names'])

In [13]:
from torch import nn
# 取得模型的classifier.Linear層
model.classifier[1] = nn.Linear(in_features=1280,
                                out_features=18,
                                bias=True).to(device)

In [14]:
# 把除了Sequential以外的參數都列為不可訓練
for param in model.features.parameters():
  param.requires_grad = False

In [15]:
cost_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [16]:
from tqdm.auto import tqdm

epochs = 20 #把資料看完10次
for epoch in tqdm(range(epochs)):
  print(f'Epoch: {epoch}\n-------')

  train_step(train_dataloader, model, cost_fn, optimizer, device)
  test_step(test_dataloader, model, cost_fn, device)

  0%|          | 0/20 [00:00<?, ?it/s]

Epoch: 0
-------


/usr/local/lib/python3.10/dist-packages/PIL/Image.py:996: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Train Cost: 2.3429, Train Acc:52.71
Test Cost:1.9299, Test Acc:69.45
Epoch: 1
-------
Train Cost: 1.5915, Train Acc:72.52
Test Cost:1.4946, Test Acc:72.42
Epoch: 2
-------
Train Cost: 1.2285, Train Acc:76.72
Test Cost:1.2504, Test Acc:76.01
Epoch: 3
-------
Train Cost: 1.0252, Train Acc:79.91
Test Cost:1.0966, Test Acc:77.36
Epoch: 4
-------
Train Cost: 0.9015, Train Acc:80.60
Test Cost:0.9722, Test Acc:79.08
Epoch: 5
-------
Train Cost: 0.8027, Train Acc:83.60
Test Cost:0.9462, Test Acc:79.75
Epoch: 6
-------
Train Cost: 0.7381, Train Acc:84.44
Test Cost:0.8773, Test Acc:81.08
Epoch: 7
-------
Train Cost: 0.6777, Train Acc:85.48
Test Cost:0.8275, Test Acc:80.68
Epoch: 8
-------
Train Cost: 0.6169, Train Acc:86.47
Test Cost:0.7953, Test Acc:81.36
Epoch: 9
-------
Train Cost: 0.5752, Train Acc:87.25
Test Cost:0.7735, Test Acc:80.68
Epoch: 10
-------
Train Cost: 0.5460, Train Acc:87.80
Test Cost:0.7530, Test Acc:81.34
Epoch: 11
-------
Train Cost: 0.5345, Train Acc:87.52
Test Cost:0.7298

In [17]:
img = Image.open('luffy.webp').convert('RGB')
img = efficientnet_b1_transforms(img)
# img.shape
img = img.reshape(-1,3,240,240)
model.eval()
with torch.inference_mode():
  y_pred = model(img.to(device))
# y_pred = torch.softmax(y_pred, dim=1)
y_pred.argmax(dim=1)

tensor([9], device='cuda:0')

In [18]:
img = Image.open('nami2.jpeg').convert('RGB')
img = efficientnet_b1_transforms(img)
# img.shape
img = img.reshape(-1,3,240,240)
model.eval()
with torch.inference_mode():
  y_pred = model(img.to(device))
# y_pred = torch.softmax(y_pred, dim=1)
y_pred.argmax(dim=1)

tensor([11], device='cuda:0')

In [19]:
img = Image.open('mihawk2.jpeg').convert('RGB')
img = efficientnet_b1_transforms(img)
# img.shape
img = img.reshape(-1,3,240,240)
model.eval()
with torch.inference_mode():
  y_pred = model(img.to(device))
# y_pred = torch.softmax(y_pred, dim=1)
y_pred.argmax(dim=1)

tensor([10], device='cuda:0')

In [21]:
torch.save(obj=model.state_dict(),f = 'model/pytorch_one_piece_full_classfi.pth')